In [1]:
import numpy as np
import pandas as pd
import chardet
from datetime import date
from scipy.stats import norm
from scipy.stats import chi2
import matplotlib.pyplot as plt

## 1. Чтение, обработка данных

In [2]:
def compute_and_add_days(df, name):
    df = df.reset_index(drop = True)
    n_cols = df.shape[1]
    n_rows = df.shape[0]
    ret = df;
    uncorrects = [[] , 0];
    row_days = []
    row0 = df.loc[0];
    start = date(row0[0], row0[1], row0[2])
    
    for i in range(0, n_rows):   
        row = ret.loc[i]
        if ((row[2] < 1) or (row[2] > 31)): #or (row[1] < 1) or (row[1] > 12)):
            ret = ret.drop(i, axis = 0)
            continue 
            
        curr = date(row[0], row[1], row[2])
        row_days.append((curr - start).days)
        
    ret[name] = row_days
    ret = ret.reset_index(drop = True)
    ret = ret.sort_values(by=[name])
    ret = ret.reset_index(drop=True)
    return ret

## 2. Разбиение данных на две выборки по времени и по колличеству заявок в промежутках времени

In [3]:
def first_method_reduce(df, step):
    n_cols = df.shape[1]
    n_rows = df.shape[0]
    
    ret = []
    i = 0
    while (i < (n_rows - 1)):
        row = [df.loc[i][n_cols - 1], i]
        while ((df.loc[i + 1][n_cols - 1] - df.loc[i][n_cols - 1] <= step) and (i < (n_rows - 2))):
            i += 1            
        i += 1
        ret.append([row[0], i - row[1]])
    ret[-1][1] += 1
    ret = np.array(ret)
    return (ret)

def second_method_reduce(df, step):
    n_cols = df.shape[1]
    n_rows = df.shape[0]
    
    ret = []
    i = 0
    while (i < (n_rows - 1)):
        row = [df.loc[i][n_cols - 1], i]
        while ((df.loc[i + 1][n_cols - 1] - row[0] < step) and (i < (n_rows - 2))):
            i += 1            
        i += 1
        ret.append([row[0], i - row[1]])
    ret[-1][1] += 1
    ret = np.array(ret)
    return (ret)



def third_method_reduce(df, step_params):
    step_0 = step_params[0]
    step_1 = step_params[1]
    step_2 = step_params[2]
    d_step = step_params[3]
    
    df = first_method_reduce(df, step_0)
    ret = []
    i = 0
    s = 0
    k = 0
    while (s != -1):
        s = -1
        n_rows = len(df)
        
        s_tmp1 = 1000000
        s_tmp2 = 1000000
        i = 0
        while (i < n_rows-2):
            time_req_1 = df[i]
            time_req_2 = df[i + 1]
            if (time_req_1[1] <= d_step and time_req_2[1] == d_step+1 and time_req_2[0] - time_req_1[0] < step_1):
                s_tmp1 = i
                break
            i += 1
        
        i = 0
        while (i < n_rows-2):
            time_req_1 = df[i]
            time_req_2 = df[i + 1]
            if (time_req_1[1] <= d_step and time_req_2[1] <= d_step and time_req_2[0] - time_req_1[0] < step_2):
                s_tmp2 = i
                break
            i += 1
        s = min(s_tmp1, s_tmp2)
        if (s > 0 and s < 1000000):
            i = 0
            tmp_df = []
            while (i <= s):
                tmp_df.append(df[i])
                i+=1
            if (i == s + 1):
                tmp_df[s][0] = df[i][0]
                tmp_df[s][1] += df[i][1]
            while (i < n_rows-1):
                tmp_df.append(df[i + 1])
                i+=1
            df = np.array(tmp_df)
        else: 
            break
    return (df)

def make_reduce(sample, method, steps, country):
    reduce = method(sample, steps)
    #print(reduce)
    print("Reduce data for", country, "with days from start, shape: " , len(reduce))
    time_smp = np.diff(reduce[:,0])
    requests_smp = reduce[:, 1]
    print("time intervals = ")
    print(time_smp)
    print("requests = ")
    print(requests_smp)
    return time_smp, requests_smp

## 3. Проверка независимости случайных величин в выборках с помощью критерия Валлиса - Мура

In [4]:
def num_sign_changes(arr):
    i = 0
    sign = arr[0]
    count = 0
    while i < len(arr) - 1:
        if arr[i] != 0:
            if sign == 0:
                sign = arr[i]
            elif sign != arr[i]:
                count += 1
                sign = arr[i]
        i +=1
    return count

def Wallis_Murr_number(sample):
    sample_diff = np.diff(sample)
    gamma = num_sign_changes(np.sign(sample_diff)) - 2
    n = len(sample)
    return np.abs((gamma - (2*n - 7)/3)*np.sqrt(90/ (16*n - 29))) 

def Wallis_Murr_crit(sample, alfa, str):
    Z1 = Wallis_Murr_number(sample)
    f = norm(0, 1)
    treshold_val = f.ppf(alfa/2)
    if (Z1 <= -treshold_val):
        print(str, "is undependet", round(Z1, 2), " <= ", -round(treshold_val,2))
    else:
        print(str, "is dependet", round(Z1, 2), " > ", -round(treshold_val, 2))
    

## 4. Оценка параметров распределения для выборки событий и критерий хи2

In [2]:
def func_sum(inter, func):
    sum = 0 
    for i in np.arange(inter[0], inter[1] + 1): 
        sum += func(i)
    return sum

def divide_time(time, r, func):
    tmp = np.bincount(time)
    interv = len(time)//r
    intervp = 1/r
    m = np.zeros(r)
    p = np.zeros(r)
    j = 0
    for i in range(0, r):
        if (j < len(tmp)):
            m[i] += tmp[j]
            p[i] += func(j)
            j += 1
        while (p[i] < intervp) and (j < len(tmp)):
            intervp = (1-sum(p))/(r - i)
            m[i] += tmp[j]
            p[i] += func(j)
            j += 1
    p[len(p) - 1] = 1 - sum(p[0:len(p) - 1])
    return m, p

def Chi_squre_number(m, p):
    ret = 0
    r = len(m)
    n = np.sum(m)
    for i in range(0, r):
        ret += (m[i] - n*p[i])**2 / n*p[i]
    return ret

def Chi_2_crit(sample, r, P, alfa, num_param):
    m, p = divide_time(sample, r, P)    
    chi_stat = Chi_squre_number(m, p)
    chi_2 = chi2(r - 1 - num_param)
    print("Chi2 degree: ", r - 1 - num_param)
    chi_2_tres1 = chi_2.ppf(1 - alfa)
    print("Chi2 statistic:         ", round(chi_stat, 2))
    print("Chi2 treshould val right:", round(chi_2_tres1, 2))

In [6]:
def plot_hist(sample, r_bound, P, ax, title, n=0):
    x = np.arange(0, r_bound)
    Fx = []
    
    for i in x:
        Fx.append(P(i))
    if (n == 0):
        n = round(np.math.sqrt(len(sample)))

    bins = np.linspace(sample.min(), r_bound, n) 
    ax.hist(sample, bins, density=1)
    ax.plot(x, Fx, color="red")
    ax.legend([r'$f_\xi(x)$', r'$histogram$'], fontsize=20)
    ax.set_title("Hist Plot " + title , fontsize = 15)
    ax.grid()

## 5. Распределения о подбор параметров

In [7]:
def Poisson_param(param):
    def Poisson(k):
        fact = 1
        if k < 1:
            return 0
        else:
            for i in range(1, k):
                fact *= param/i
            return np.exp(-param) * fact
    return Poisson

def geom_param(param):
    def geom(k):
        if k < 1:
            return 0
        else:
            return np.power((1-param), k-1) * param
    return geom

def mix_distrib_param(p, func_1, func_2):
    def mix_distrib(k):
        return p*func_1(k) + (1-p)*func_2(k)
    return mix_distrib

from scipy.optimize import fsolve

def GMM_Poisson(sample):
    a1 = np.average(sample)
    return a1 - 1

def GMM_geom(sample):
    a1 = np.average(sample)
    return 1/a1

def GMM_mix_distrib(sample, a2, a3):
    a1 = np.average(sample)
    equations = lambda x: (x*a2 + (1 - x)*a3 - a1)
    return fsolve(equations, 0.9)

def process_sample(sample, delimiter, missed_vals):
    sample1 = (np.sort(sample))[0:len(sample) - missed_vals]
    ret1 = sample1[sample1 <= delimiter]
    ret2 = sample1[sample1 > delimiter]
    return ret1, ret2

In [8]:
def shift_exp_param(param):
    h, sigm = param
    def shift_exp(t):
        if t <= h:
            return 0
        else:
            return 1/sigm * np.exp((h - t) / sigm)
    def F(t):
        if t <= h:
            return 0
        else:
            return 1 - np.exp((h - t) / sigm)
    return shift_exp, F

from scipy.optimize import fsolve

def GMM_shift_exp(sample, missed_vals):
    tmp = np.sort(sample)[0:len(sample)-missed_vals]
    a1 = np.average(tmp)
    a2 = np.std(tmp)
    return [ a1 - a2, a2]

def divide_time1(time, r, func):
    tmp1 = np.bincount(time)
    interv = len(time)//r
    intervp = 1/r
    m = np.zeros(r)
    p = np.zeros(r + 1)
    bounds = np.zeros(r + 1)
    j = 0
    h = 0.25
    interval_sum = 0
    for i in range(1, r + 1):
        while (p[i] < intervp) and (func(interval_sum) < func(max(time))):
            interval_sum +=h
            p[i] = func(interval_sum) - func(bounds[i - 1])   
        bounds[i] = interval_sum
        tmp = time[time < bounds[i]]
        m[i - 1] = len(tmp[tmp >= bounds[i - 1]])
    p[r] = 1 -sum(p[:r])
    p = p[1:]
    return m, p

def Chi_2_crit1(sample, r, P, alfa, num_param):
    m, p = divide_time1(sample, r, P)
    
    chi_stat = Chi_squre_number(m, p)
    chi_2 = chi2(r - 1 - num_param)
    print("Chi2 degree: ", r - 1 - num_param)
    chi_2_tres1 = chi_2.ppf(1 - alfa)
    print("Chi2 statistic:         ", round(chi_stat, 2))
    print("Chi2 treshould val right:", round(chi_2_tres1, 2))